# Initialization

In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_setups.frame_tools import *
SET = 'MKM'
data_manager, set_data = set_up(log_lvl=LogLvl.SPARSE, target_set=SET)

# Objects

In [ ]:
from core.data_graphing.PlotterHelper import PlotterHelper
from core.data_graphing.PickOrderAnalyzer import PickOrderAnalyzer
from core.data_graphing.utils.color.plotting_config import DefaultPlotConfigs
from core.data_graphing.utils.funcs import prettify_frame
import core.wubrg.alias_mappings as color_alias

ROLL = 3
DERIVS = 0
SRT_TRG = 'ALSA Change'
SAVE = False

In [ ]:
from core.wubrg import get_color_identity

class FramedDataFuncs:
    def __init__(self, DATA):
        self._DATA = DATA
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._DATA.SET
    
    @property
    def FULL_SET(self):
        """The full name of the draft set."""
        return self._DATA.FULL_SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._DATA.FORMAT
    
    @property
    def SHORT_FORMAT(self):
        """The shorthand of the format type."""
        return self._DATA.FULL_FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None, *, save=False):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll, save=save)

    def plot_archetype_winrate_history(self, plot_config, roll=None, derivs=0):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(plot_config, roll, derivs)       
        
    def get_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, *, save=False):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, color_count, roll, save=save)

    def plot_archetype_playrate_history(self, plot_config, color_count=0, roll=None, derivs=0):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(plot_config, color_count, roll, derivs)
    
    def card_relative_winrates(self, deck_colors, win_rate_col, filter_option=None):
        return self._ARCH_FUNCS.card_relative_winrates(deck_colors, win_rate_col, filter_option)
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name, color_cols=None):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name, color_cols)
    
    def stat_archetype_performance(self, stat_name, color_cols=None):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols)
    
    def compare_card_evaluations(self, start_date=None, end_date=None):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df
        
        if start_date is None:
            metadata = SetMetadata.get_metadata(self.SET)
            start_date = metadata.RELEASE_DATE
            
        if end_date is None:
            end_date = date.today() - timedelta(days=1)

        first = inner_func(date=start_date)
        last = inner_func(date=end_date)
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['ALSA Change'] = first['ALSA'] - last['ALSA']
        diff['ATA Change'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'ALSA Change', 'ATA', 'ATA Change', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True)
        filters = list()
        if card_color:
            filters.append(card_color_filter(card_color))
        if card_rarity:
            filters.append(rarity_filter(card_rarity))
        frame = filter_frame(frame, column, filters)#, asc)

        if play_lim is not None:
            #TODO: Fix this. 
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)
    
    def get_top_performers(self, card_color=None, common_cnt=10, uncommon_cnt=5, deck_color='', play_lim=None):
        play_lim = play_lim or round(self.card_frame(deck_color=deck_color, summary=True)['# GP'].max() * 0.005)
        commons = self.get_top('GIH WR', count=common_cnt, card_color=card_color, card_rarity='C', deck_color=deck_color, play_lim=play_lim)
        uncommons = self.get_top('GIH WR', count=uncommon_cnt, card_color=card_color, card_rarity='U', deck_color=deck_color, play_lim=play_lim)
        top_cards = pd.concat([commons, uncommons])
        top_cards = top_cards.drop(['Type Line', 'Supertypes', 'Types', 'Subtypes', 'Power', 'Toughness'], axis=1)
        return top_cards.sort_values('GIH WR', ascending=False)

In [ ]:
from core.data_graphing import ROLL
from core.wubrg import COLOR_PAIRS, COLOUR_GROUPINGS, get_color_subsets
from core.data_fetching.utils.consts import COLOR_COUNT_REVERSE_MAP, COLOR_COUNT_SHORTHAND, COLOR_COUNT_SHORTHAND_MAP

class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    @staticmethod
    def _get_play_stat_frame(frame, col, roll=None, aggfunc=None):
        if roll is None: roll = ROLL
        if aggfunc is None: aggfunc = np.sum
        ret = frame[[col]]
        ret.reset_index(inplace=True, level=1)
        ret = ret.pivot_table(values=col, index=ret.index, columns='Name', dropna=False, aggfunc=aggfunc)
        ret = ret.rolling(window=roll, min_periods=1, center=True).mean()
        ret.columns.names = [col]
        return ret

    def _get_all_stat(self, stat, roll=None, aggfunc=None):
        archetypes = self._get_play_stat_frame(self._DATA.deck_archetype_frame(), stat, roll, aggfunc)
        archetypes = archetypes[[color for color in COLOR_COMBINATIONS if color in archetypes.columns]]

        groups = self._get_play_stat_frame(self._DATA.deck_group_frame(), stat, roll, aggfunc)
        groups.rename(columns=COLOR_COUNT_SHORTHAND_MAP, inplace=True)
        groups = groups[COLOR_COUNT_SHORTHAND]

        return pd.concat([groups, archetypes], axis=1)


    def get_winrates(self, roll=None):
        wins = self._get_all_stat('Wins', roll, np.sum)
        games = self._get_all_stat('Games', roll, np.sum)
        frame = (wins / games) * 100
        frame.columns.names = ['Avg. Win %']
        return frame


    def get_playrates(self, num_colors=0, roll=None):
        #TODO: Remove num_colors, and handle bubbling chnages from that.
        games = self._get_all_stat('Games', roll, np.sum)
        
        if num_colors == 0:
            frame = games.div(games['ALL'], axis='rows') * 100
            frame.columns.names = ['% of Decks']
            return frame

        true_name = COLOR_COUNT_REVERSE_MAP[num_colors].title()
        summ_id = COLOR_COUNT_SHORTHAND_MAP[true_name]

        frame = games.div(games[summ_id], axis='rows') * 100
        frame.columns.names = [f'% of {num_colors}C Decks']
        return frame
   
    def get_archetype_winrate_history(self, color_filter=None, roll=None, *, save=False):
        if roll is None: roll = ROLL
            
        frame = self.get_winrates(roll)
       
        if isinstance(color_filter, str):
            # TODO: Make this more flexible.
            #col_filt = ['ALL', f'{len(color_filter)}C'] + [col for col in COLOR_PAIRS if color_filter in col]  
            col_filt = ['ALL', '2C'] + [col for col in COLOR_PAIRS if color_filter in col]  
            frame = frame[col_filt]
        elif isinstance(color_filter, list):
            frame = frame[color_filter]
            
        if save:
            PlotterHelper(self._DATA).frame_to_png(frame, "archetype_winrate_table.png")
        
        return frame        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, *, save=False):
        if roll is None: roll = ROLL
            
        frame = self.get_playrates(color_count, roll)

        if isinstance(color_filter, str):
            col_filt = [col for col in COLOR_PAIRS if color_filter in col]
            frame = frame[col_filt]
        if isinstance(color_filter, list):
            frame = frame[color_filter]
            
        if save:
            PlotterHelper(self._DATA).frame_to_png(frame, "archetype_playrate_table.png")

        return frame
    
    def plot_archetype_winrate_history(self, plot_config, roll=None, derivs=0, save=False):
        if roll is None: roll = ROLL
        color_filter = plot_config.column_list

        data = self.get_archetype_winrate_history(color_filter, roll)
        for _ in range(0, derivs):
            data = data.diff()
        data.index = [idx[5:] for idx in data.index]
        colors = str(color_filter)  #TODO: Make this cleverer at showing filter.

        plot_help = PlotterHelper(self._DATA, plot_config)
        fig, ax = plot_help.new_single_plot('Archetype Winrates', width=16, height=8)
        plot_help.accredit(y=0.02, x=0.51)
        plot_help.desc_note(colors=colors, roll=roll, y=0.94, x=0.51)
        
        plot_help.set_labels(x_label="Date", y_label="Win Rate")
        plot_help.set_data(data, color_filter)
        
        if save:
            plot_help.save_fig(f"{plot_config.file_name_prefix}_win_rates_{roll}day_avg.png", "Metagame")

    def plot_archetype_playrate_history(self, plot_config, color_count=0, roll=None, derivs=0, save=False):
        if roll is None: roll = ROLL
        color_filter = plot_config.column_list

        data = self.get_archetype_playrate_history(color_filter, color_count, roll)
        for _ in range(0, derivs):
            data = data.diff()
        data.index = [idx[5:] for idx in data.index]
        
        
        plot_help = PlotterHelper(self._DATA, plot_config)
        fig, ax = plot_help.new_single_plot('Archetype Playrates', width=16, height=8)
        plot_help.accredit(y=0.02, x=0.51)
        plot_help.desc_note(colors=color_filter, roll=roll, y=0.94, x=0.51)
        
        plot_help.set_labels(x_label="Date", y_label="Percent of Metagame")
        plot_help.set_data(data, color_filter)
        
        if save:
            plot_help.save_fig(f"{plot_config.file_name_prefix}_play_rates_{roll}day_avg.png", "Metagame")
            
    
    def card_relative_winrates(self, deck_colors=None, win_rate_col='GIH WR', filter_option=None):
        deck_colors = get_color_identity(deck_colors)
        
        # Get the relevant list of cards, and then trim it down to the relevant colours.
        sub_frame = self._DATA.card_frame(deck_color=deck_colors)
        sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice(None), get_color_slice(deck_colors), get_name_slice(None)]
        sub_frame = sub_frame.reset_index(level=1)
        if deck_colors:
            sub_frame = sub_frame[sub_frame['Cast Color'].isin(get_color_subsets(deck_colors))]

        # Get the winrates for the cards and average winrate of the archetype, then re-center the card winrates with it. 
        win_frame = self._get_play_stat_frame(sub_frame, win_rate_col, roll=1, aggfunc=np.mean)
        avg_frame = self.get_archetype_winrate_history(deck_colors)
        target_avg = deck_colors if deck_colors else 'ALL'
        ret_frame = win_frame.sub(avg_frame[target_avg], axis='rows').T

        # Get the games played from the subset of cards and use it to re-weight the win rates per day, to calculate an accurate average.
        games_frame = self._get_play_stat_frame(sub_frame, '# GP', roll=1, aggfunc=np.mean).T
        ret_frame[f'AVG'] = (ret_frame * games_frame).sum(axis=1) / games_frame.sum(axis=1)
        ret_frame['# GP'] = games_frame.sum(axis=1)

        # Sort by most winning first.
        ret_frame = ret_frame.sort_values(f'AVG', ascending=False).T
        return ret_frame[::-1]

In [ ]:
from core.data_graphing import ROLL
from core.wubrg import COLOR_COMBINATIONS, COLOR_PAIRS
from core.data_fetching.utils.consts import STAT_COL_NAMES

class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._COLOR_IDX = 0
        
    def _shorten_data(self, card_name, roll, cols, colors=''):
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[cols]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None, plot_config=None):
        if roll is None: roll = ROLL
        plot_config = plot_config or DefaultPlotConfigs.STATS.plot_config
        rolling = self._shorten_data(card_name, roll, plot_config.column_list, colors=colors)
        if rolling.empty:
            print(f"""Could not find data for "{card_name}". Please make sure it is spelled correctly, or you are accessing the right set.""")
            return False
        
        plot_help = PlotterHelper(self._DATA, plot_config)
        # TODO: Better center the title.
        fig, ax = plot_help.new_quad_plot(card_name)
        plot_help.accredit(y=0.1)
        plot_help.desc_note(colors=colors, roll=roll, x=0.505)
                
        plot_help.set_labels(y_label="Win Percent", g_x=0, g_y=0)
        plot_help.set_data(rolling, ['GP WR', 'GIH WR', 'OH WR', 'GD WR'], g_x=0, g_y=0)
        
        plot_help.set_labels(y_label="Pick Number", g_x=0, g_y=1)
        plot_help.set_data(rolling, ['ALSA', 'ATA'], inv_y=True, g_x=0, g_y=1)
        
        plot_help.set_labels(x_label="Date", y_label="# of Games", g_x=1, g_y=0)
        plot_help.set_data(rolling, ['# GP', '# GIH', '# OH', '# GD'], g_x=1, g_y=0)

        plot_help.set_labels(x_label="Date", y_label="# of Cards", g_x=1, g_y=1)
        plot_help.set_data(rolling, ['# Seen', '# Picked'], g_x=1, g_y=1)
        
        if colors:
            plot_help.save_fig(f"pcs_{card_name}_{colors}.png", "Summary")
        else:
            plot_help.save_fig(f"pcs_{card_name}.png", "Summary")
        
        return True
    
    
    def plot_pick_stats(self, card_name, roll=None, plot_config=None):
        if roll is None: roll = ROLL        
        plot_config = plot_config or DefaultPlotConfigs.STATS.plot_config
        taken_data = self._shorten_data(card_name, roll, ['ALSA', 'ATA'])
        if taken_data.empty:
            print(f"""Could not find data for "{card_name}". Please make sure it is spelled correctly, or you are accessing the right set.""")
            return
        
        plot_help = PlotterHelper(self._DATA, plot_config)
        fig, ax = plot_help.new_single_plot(card_name)
        plot_help.accredit(y=-0.01)
        plot_help.desc_note(roll=roll, y=0.96)
        
        plot_help.set_labels(x_label="Date", y_label="Pick Number")
        plot_help.set_data(taken_data, ['ALSA', 'ATA'], inv_y=True)
        
        plot_help.save_fig(f"pps_{card_name}.png", "Pick Stats")


    def card_archetype_performance(self, card_name, color_cols=None):
        frame = self._DATA.card_frame(card_name, summary=True).T
        frame.loc[STAT_COL_NAMES].T
        if color_cols is not None:
            ret = ret[color_cols]
        return frame
    
    
    def stat_archetype_performance(self, stat_name, color_cols=None):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

In [ ]:
examiner = FramedDataFuncs(set_data.BO1)
frame = examiner.card_frame(deck_color='', summary=True)
pd.set_option('display.max_rows', len(frame))
pd.set_option('display.max_columns', len(frame.columns))

# Data Overview

## Top Performing Cards

### All Cards

In [ ]:
examiner.get_top_performers(None, 25, 10, '', None)

### White

In [ ]:
examiner.get_top_performers('W')

### Blue

In [ ]:
examiner.get_top_performers('U')

### Black

In [ ]:
examiner.get_top_performers('B')

### Red

In [ ]:
examiner.get_top_performers('R')

### Green

In [ ]:
examiner.get_top_performers('G')

### Two-Colour

In [ ]:
examiner.get_top_performers(['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG'])

## Archetype Playrate & Winrate History

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.TWO_COLOR, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.TWO_COLOR, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.ALLIED, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.ALLIED, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.ENEMY, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.ENEMY, roll=ROLL)

## Card Pick Order Changes

In [ ]:
analyzer = PickOrderAnalyzer(examiner.DATA, SRT_TRG)
styled_frames = analyzer.gen_card_evaluation_summary()

In [ ]:
styled_frames["ContestedCommons"]

In [ ]:
styled_frames["UncontestedCommons"]

In [ ]:
styled_frames["TopPerformingCommons"]

In [ ]:
styled_frames["ContestedUncommons"]

In [ ]:
styled_frames["UncontestedUncommons"]

In [ ]:
styled_frames["TopPerformingUncommons"]

In [ ]:
wheelable_cards = analyzer.get_wheelable_summary(save=SAVE)
wheelable_cards['']

In [ ]:
raise Exception('Stopping Auto-Run!')

# Color Performance Analysis

## ???

In [ ]:
analyzer = PickOrderAnalyzer(examiner.DATA, SRT_TRG)
wheelable_cards = analyzer.get_wheelable_summary(save=SAVE)
wheelable_iterator = iter(wheelable_cards.values())

In [ ]:
next(wheelable_iterator)

In [ ]:
examiner.get_top_performers('', deck_color='GW')

In [ ]:
plot_config = DefaultPlotConfigs.RED
examiner.plot_archetype_winrate_history(plot_config, roll=ROLL, derivs=DERIVS)
examiner.plot_archetype_playrate_history(plot_config, roll=ROLL, derivs=DERIVS)

In [ ]:
raise Exception('Stopping Auto-Run!')

# Examine Data

In [ ]:
examiner = FramedDataFuncs(set_data.BO1)

In [ ]:
prettify_frame(examiner.stat_archetype_performance('GIH WR'))

In [ ]:
prettify_frame(examiner.card_relative_winrates('WR', 'GIH WR'))

In [ ]:
prettify_frame(examiner.card_archetype_performance("Novice Inspector"))

In [ ]:
raise Exception('Stopping Auto-Run!')